# SparkR (R on Spark)

## Overview

- SparkR은 R에서 Apache Spark을 사용하기 위한 가벼운 진입점을 제공하는 R 패키지임.
- SparkR은 selection, filtering, aggregation etc. (similar to R data frames, dplyr)와 같은 동작을 지원하는 distributed data frame을 지원함.
- SparkR은 MLib을 사용해서  distributed machine learning 지원함.

## SparkR DataFrames

- Spark에서의 DataFrame은 이름이 있는 clomn들로 구성된  distributed collection으로 관계형데이터베이스의 테이블 또는 R에서의 date frame과 개념상 비슷함.
- DataFrame은 구조화된 date file, Hive의 table, 외부 데이터베이스 또는 local R data frame과 같이 다양한 소스로부터 만들어질 수 있음.

### Starting Up: SparkContext, SQLContext

- SparkR에서의 시작포인트는 R에서 Spark cluster에 접속할 수 있도록  SparkContext 생성하는 것임.
- SparkContext 생성은 sparkR.init() 함수를 사용하고, DataFrames을 가지고 작업하기 위해서는 SQLContext도 필요함.

In [ ]:
sc <- sparkR.init("local[*]")
sqlContext <- sparkRSQL.init(sc)

### Starting Up from RStudio

- R 프로그램이 RStudio, R shell, Rscript 또는  other R IDEs로부터 spark cluster에 접속하기 위해서는 
    - 환경변수 SPARK_HOME을 설정
    - SparkR package을 로딩
    - sparkR.init()을 호출할때 여러가지 설정을 함.
    - sparkEnvir 로 내부적으로 JVM호출할때 옵션이 설정.
    
- sparkEnvir 설정


|Property Name  	              | Property group	     | spark-submit equivalent
|---------------------------------|----------------------|-------------------------
|spark.driver.memory	          | ApplicationProperties| --driver-memory
|spark.driver.extraClassPath	  | Runtime Environment  |	--driver-class-path
|spark.driver.extraJavaOptions	  | Runtime Environment  |	--driver-java-options
|spark.driver.extraLibraryPath	  | Runtime Environment  |	--driver-library-path



In [1]:
if (nchar(Sys.getenv("SPARK_HOME")) < 1) {
  Sys.setenv(SPARK_HOME = "/home/spark")
}

library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))
sc <- sparkR.init(master = "local[*]", sparkEnvir = list(spark.driver.memory="2g"))
sqlContext <- sparkRSQL.init(sc)


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var

The following objects are masked from ‘package:base’:

    colnames, colnames<-, endsWith, intersect, rank, rbind, sample,
    startsWith, subset, summary, table, transform



Launching java with spark-submit command /usr/local/spark/bin/spark-submit   --driver-memory "2g" sparkr-shell /tmp/RtmpsRJlr8/backend_port1ddd11431940 


### Creating DataFrames

- SQLContext를 가지고, local R data frame, Hive table, 기타 다른 data source로부터 DataFrame을 생성해보자.

### From local data frames

- R에 내장된 faithful 데이터셋을 읽어보자, faithful은 미국의 국립공원에 있는 간헐천이 폭발지속시간(eruptions, min) 과 대기 시간(waiting, min)

In [2]:
summary( faithful )
head(faithful)

   eruptions        waiting    
 Min.   :1.600   Min.   :43.0  
 1st Qu.:2.163   1st Qu.:58.0  
 Median :4.000   Median :76.0  
 Mean   :3.488   Mean   :70.9  
 3rd Qu.:4.454   3rd Qu.:82.0  
 Max.   :5.100   Max.   :96.0  

,eruptions,waiting
1,3.6,79.0
2,1.8,54.0
3,3.333,74.000
4,2.283,62.000
5,4.533,85.000
6,2.883,55.000


In [3]:
df <- createDataFrame(sqlContext, faithful)
head(df)

,eruptions,waiting
1,3.6,79.0
2,1.8,54.0
3,3.333,74.000
4,2.283,62.000
5,4.533,85.000
6,2.883,55.000


#### From Data Sources

- 데이터 소스로부터 DataFrame을 생성하는 일반적인 방법은 read.df()함수를 사용하는것임.
- 이 함수는 JSON과 Parquet 파일을 지원

In [2]:
people <- read.df(sqlContext, "/usr/local/spark/examples/src/main/resources/people.json", "json")
head(people)

,age,name
1,NA,Michael
2,30,Andy
3,19,Justin


In [3]:
printSchema(people)

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)


- data sources API들을 사용해서 여러가지 포맷으로  DataFrame을 저장 가능함.
- write.df()함수로 Parquet 파일로 저장해보자.

In [4]:
write.df(people, path="people.parquet", source="parquet", mode="overwrite")

NULL

#### From Hive tables

- Hive tables로부터 DataFrame을 생성할 수 있음.  Hive가 설치가 필요해서 여기서는 코드만 보자.

In [ ]:
# sc is an existing SparkContext.
hiveContext <- sparkRHive.init(sc)

sql(hiveContext, "CREATE TABLE IF NOT EXISTS src (key INT, value STRING)")
sql(hiveContext, "LOAD DATA LOCAL INPATH 'examples/src/main/resources/kv1.txt' INTO TABLE src")

# Queries can be expressed in HiveQL.
results <- sql(hiveContext, "FROM src SELECT key, value")

# results is now a DataFrame
head(results)

### DataFrame Operations

- DataFrames은 구조화된 데이터 프로세싱을 위해서 여러가지 함수를 지원함.

#### Selecting rows, columns

In [8]:
df <- createDataFrame(sqlContext, faithful)
df

DataFrame[eruptions:double, waiting:double]

In [9]:
head(select(df, df$eruptions))

,eruptions
1,3.6
2,1.8
3,3.333
4,2.283
5,4.533
6,2.883


In [10]:
head(select(df, "eruptions"))

,eruptions
1,3.6
2,1.8
3,3.333
4,2.283
5,4.533
6,2.883


In [11]:
head(filter(df, df$waiting < 50))

,eruptions,waiting
1,1.75,47.00
2,1.75,47.00
3,1.867,48.000
4,1.75,48.00
5,2.167,48.000
6,2.1,49.0


#### Grouping, Aggregation

In [12]:
head(summarize(groupBy(df, df$waiting), count = n(df$waiting)))

,waiting,count
1,81,13
2,60,6
3,93,2
4,68,1
5,47,4
6,80,8


In [13]:
waiting_counts <- summarize(groupBy(df, df$waiting), count = n(df$waiting))
head(arrange(waiting_counts, desc(waiting_counts$count)))

,waiting,count
1,78,15
2,83,14
3,81,13
4,77,12
5,82,12
6,84,10


#### Operating on Columns

In [14]:
df$waiting_secs <- df$waiting * 60
head(df)

,eruptions,waiting,waiting_secs
1,3.6,79.0,4740.0
2,1.8,54.0,3240.0
3,3.333,74.000,4440.000
4,2.283,62.000,3720.000
5,4.533,85.000,5100.000
6,2.883,55.000,3300.000


### Running SQL Queries from SparkR

- DataFrame 은 Spark SQL에서 임시 테이블을 만들고, SQL문을 실행할 수 있음.

In [17]:
people <- read.df(sqlContext, "/usr/local/spark/examples/src/main/resources/people.json", "json")
head(people)

,age,name
1,NA,Michael
2,30,Andy
3,19,Justin


In [18]:
registerTempTable(people, "people")

In [19]:
teenagers <- sql(sqlContext, "SELECT name FROM people WHERE age >= 13 AND age <= 19")
head(teenagers)

,name
1,Justin


## Machine Learning

- SparkR은 glm()함수를 사용해서 DataFrames을 가지고 generalized linear models을 적합시킬 수 있음.
- SparkR의 glm()함수의 분포 가정을 gaussian(정규분포)와 binomial(이항분포)를 지원함.
- 사용법은 R의 glm()함수와 비슷함.


- Spark 2.0에서는  아래와 같이 추가 지원함.
    - “gaussian”, “binomial”, “poisson” and “gamma” 까지 지원
    - Survival Regression
    - Naive Bayes Model
    - KMeans Model
    - Model 저장  
    
### Gaussian GLM model

In [21]:
df <- createDataFrame(sqlContext, iris)
head(df)

Warning message:
In FUN(X[[i]], ...): Use Sepal_Length instead of Sepal.Length  as column nameWarning message:
In FUN(X[[i]], ...): Use Sepal_Width instead of Sepal.Width  as column nameWarning message:
In FUN(X[[i]], ...): Use Petal_Length instead of Petal.Length  as column nameWarning message:
In FUN(X[[i]], ...): Use Petal_Width instead of Petal.Width  as column name

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa


In [22]:
model <- glm(Sepal_Length ~ Sepal_Width + Species, data = df, family = "gaussian")
summary(model)

$devianceResiduals
 Min       Max     
 -1.307112 1.412532

$coefficients
                   Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)        2.251393  0.3697543  6.08889  9.568102e-09
Sepal_Width        0.8035609 0.106339   7.556598 4.187317e-12
Species_versicolor 1.458743  0.1121079  13.01195 0           
Species_virginica  1.946817  0.100015   19.46525 0

In [23]:
predictions <- predict(model, newData = df)
head(select(predictions, "Sepal_Length", "prediction"))

,Sepal_Length,prediction
1,5.100000,5.063856
2,4.900000,4.662076
3,4.700000,4.822788
4,4.600000,4.742432
5,5.000000,5.144212
6,5.400000,5.385281


### Binomial GLM model

In [24]:
df <- createDataFrame(sqlContext, iris)
training <- filter(df, df$Species != "setosa")

model <- glm(Species ~ Sepal_Length + Sepal_Width, data = training, family = "binomial")
summary(model)

Warning message:
In FUN(X[[i]], ...): Use Sepal_Length instead of Sepal.Length  as column nameWarning message:
In FUN(X[[i]], ...): Use Sepal_Width instead of Sepal.Width  as column nameWarning message:
In FUN(X[[i]], ...): Use Petal_Length instead of Petal.Length  as column nameWarning message:
In FUN(X[[i]], ...): Use Petal_Width instead of Petal.Width  as column name

,Estimate
(Intercept),-13.046
Sepal_Length,1.902373
Sepal_Width,0.404655


## R Function Name Conflicts

- SparkR 패키지 내부의 함수와 R의 함수명이 충돌이 발생할 수 있으며, 아래와 같이 해결할 수 있음

![](sparkR_01.jpg)